In [3]:
import os
import pandas as pd
import chardet
import numpy as np
import math
import random
import nltk
nltk.download('cmudict')
from datetime import datetime
formatted_date=datetime.today().strftime("%m%d%y")
base_directory = r"C:\Users\razza\ConnTextUL_data\data\fiction_counts.csv"
data=pd.read_csv(base_directory)

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\razza\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [4]:
coca= data.copy()
coca['word'] = coca['word'].str.lower()
coca['word'] = coca['word'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
coca_filtered = coca.dropna(subset=['word'])
coca_filtered.reset_index(drop=True, inplace=True)
coca_filtered

,word,freq
0,amalfi,33
1,currents,605
2,surcos,2
3,smurfie,1
4,friges,1
...,...,...
359623,peplos,1
359624,resideth,1
359625,rogue,329
359626,litigator,39


In [5]:
word_frequency_coca=coca_filtered
word_frequency_coca.columns = ['unique_words', 'frequency']
word_frequency_coca

,unique_words,frequency
0,amalfi,33
1,currents,605
2,surcos,2
3,smurfie,1
4,friges,1
...,...,...
359623,peplos,1
359624,resideth,1
359625,rogue,329
359626,litigator,39


In [6]:
# Calculate the total word count
total_word_count = word_frequency_coca['frequency'].sum()
# Calculate the average per million value
word_frequency_coca['average_per_million'] = (word_frequency_coca['frequency'] / total_word_count) * 1e6
# Calculate the total "average_per_million" value
total_average_per_million = word_frequency_coca['average_per_million'].sum()
# Calculate the sampling probability based on "average_per_million"
word_frequency_coca['sampling_probability_per_million'] = word_frequency_coca['average_per_million'] / total_average_per_million
# Calculate the sampling probability 
word_frequency_coca['sampling_probability'] = word_frequency_coca['frequency'] / total_word_count
# Display the new DataFrame with the sampling probability
print(word_frequency_coca)

       unique_words  frequency  average_per_million  \
0            amalfi         33             0.329774   
1          currents        605             6.045849   
2            surcos          2             0.019986   
3           smurfie          1             0.009993   
4            friges          1             0.009993   
...             ...        ...                  ...   
359623       peplos          1             0.009993   
359624     resideth          1             0.009993   
359625        rogue        329             3.287743   
359626    litigator         39             0.389732   
359627  poppasquash          2             0.019986   

        sampling_probability_per_million  sampling_probability  
0                           3.297736e-07          3.297736e-07  
1                           6.045849e-06          6.045849e-06  
2                           1.998628e-08          1.998628e-08  
3                           9.993139e-09          9.993139e-09  
4             

C:\Users\razza\AppData\Local\Temp\ipykernel_16692\4272252624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_frequency_coca['average_per_million'] = (word_frequency_coca['frequency'] / total_word_count) * 1e6
C:\Users\razza\AppData\Local\Temp\ipykernel_16692\4272252624.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_frequency_coca['sampling_probability_per_million'] = word_frequency_coca['average_per_million'] / total_average_per_million
C:\Users\razza\AppData\Local\Temp\ipykernel_16692\42

In [7]:
final_datafile='probabilities_coca_'+formatted_date+'.csv'
word_frequency_coca.to_csv(final_datafile, encoding='utf-8',index=False)

In [8]:
cmu_words = nltk.corpus.cmudict.dict()
def is_in_cmu_dict(word):
    return word.lower() in cmu_words

# Function to generate word
def generate_word(words,probs):
    word=random.choices(words,weights=probs)
    return str(word[0])
def create_file(filename, rows , words,probab):
    file_name=filename
    rows=rows
    words=words
    probs=probab
    scaling_factor= rows/len(words)
    remainder = rows - math.floor(scaling_factor)
    with open(file_name, 'w') as file:
        for i in range(math.floor(scaling_factor)):
            word = generate_word(words,probs)
            while not is_in_cmu_dict(word):
                word = generate_word(words,probs)
            file.write((word) + '\n')
        for r in range(remainder):
            word_r=generate_word(words,probs)
            while not is_in_cmu_dict(word_r):
                word_r = generate_word(words,probs)
            file.write((word_r) + '\n')

In [10]:
#100k
rows=100000
file_name='COCA_'+str(rows)+'_'+formatted_date+'.csv'
words=word_frequency_coca['unique_words']
probabilities=word_frequency_coca['sampling_probability']
create_file(file_name,rows,words,probabilities)